In [1]:
import pandas as pd
import plotly.express as px
import nbformat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor




import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [2]:
DinningPred = pd.read_excel("DinningData.xlsx", sheet_name='Data for 2025 pred')


In [3]:
DinningPred.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        285 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [4]:
missing_values = DinningPred.isnull().sum()
missing_values


DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [246]:
print(DinningPred.columns)

Index(['DEP', 'Semester', 'Month', 'Year', 'Hours', 'Money Spent'], dtype='object')


In [247]:
DinningPred

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,NaN,0.0000
281,5,3,2,2025,NaN,0.0000
282,5,3,3,2025,NaN,0.0000
283,5,3,4,2025,NaN,0.0000


In [257]:
def impute_with_models(df, target_column, feature_columns, tolerance=0.1, n_neighbors=4,n_estimators=50, random_state=42):
    # Separate the data into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df.dropna(subset=[target_column])

    # If there are no missing values, return the original dataframe
    if df_missing.empty:
        print(f"No missing values for {target_column}. Skipping imputation.")
        return df, None, None, None, None, None, None, None, None  # Return None for metrics

    # Split df_not_missing into training and testing sets
    X = df_not_missing[feature_columns]
    y = df_not_missing[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit a linear regression model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the linear regression model
    lr_y_pred = lr_model.predict(X_test)

    # Calculate evaluation metrics for linear regression
    lr_mse = mean_squared_error(y_test, lr_y_pred)
    lr_mae = mean_absolute_error(y_test, lr_y_pred)
    lr_r2 = r2_score(y_test, lr_y_pred)

    print(f"Linear Regression Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {lr_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {lr_mae:.2f}")
    print(f"R-squared (R²): {lr_r2:.2f}")

    # Create and fit a KNN model
    knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the KNN model
    knn_y_pred = knn_model.predict(X_test)

    # Calculate evaluation metrics for KNN
    knn_mse = mean_squared_error(y_test, knn_y_pred)
    knn_mae = mean_absolute_error(y_test, knn_y_pred)
    knn_r2 = r2_score(y_test, knn_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {knn_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {knn_mae:.2f}")
    print(f"R-squared (R²): {knn_r2:.2f}")

    # Create and fit a Random Forest Regressor model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the model
    rf_y_pred = rf_model.predict(X_test)

    # Calculate evaluation metrics
    rf_mse = mean_squared_error(y_test, rf_y_pred)
    rf_mae = mean_absolute_error(y_test, rf_y_pred)
    rf_r2 = r2_score(y_test, rf_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {rf_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {rf_mae:.2f}")
    print(f"R-squared (R²): {rf_r2:.2f}")


    # Predict missing values using the model (choose the better model based on R²)
    X_missing = df_missing[feature_columns]
    if lr_r2 > knn_r2 and lr_r2 > rf_r2 :
        print("Using Linear Regression for imputation.")
        predicted_values = lr_model.predict(X_missing)
    elif knn_r2 > lr_r2 and knn_r2 > rf_r2:
        print("Using KNN for imputation.")
        predicted_values = knn_model.predict(X_missing)
    else:
        print("Using RF for imputation.")
        predicted_values = rf_model.predict(X_missing)

    # Create a dataset with imputed missing values
    df_filled = df.copy()
    df_filled.loc[df_missing.index, target_column] = predicted_values

    return df_filled, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2


In [258]:
target_columns_to_impute = ['Hours']
feature_columns_for_imputation = ['Year', 'Month', 'DEP', 'Semester']
linear_data = DinningPred.copy()

In [259]:
for target_column in target_columns_to_impute:
    linear_data, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2 = impute_with_models(linear_data, target_column, feature_columns_for_imputation)
    print(f"Linear Regression - MSE: {lr_mse}, MAE: {lr_mae}, R²: {lr_r2}")
    print(f"KNN - MSE: {knn_mse}, MAE: {knn_mae}, R²: {knn_r2}")
    print(f"RF - MSE: {rf_mse}, MAE: {rf_mae}, R²: {rf_r2}")

Linear Regression Evaluation metrics for Hours:
Mean Squared Error (MSE): 128528.66
Mean Absolute Error (MAE): 273.10
R-squared (R²): 0.58
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 35692.71
Mean Absolute Error (MAE): 104.51
R-squared (R²): 0.88
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 40066.74
Mean Absolute Error (MAE): 117.62
R-squared (R²): 0.87
Using KNN for imputation.
Linear Regression - MSE: 128528.65507646448, MAE: 273.10225925214013, R²: 0.5828380816448937
KNN - MSE: 35692.70673859878, MAE: 104.50530383333334, R²: 0.884153164090125
RF - MSE: 40066.74014690586, MAE: 117.6155951154167, R²: 0.8699564842410037


In [260]:
print(linear_data)


     DEP  Semester  Month  Year      Hours  Money Spent
0      1         1      9  2020  3859.9800   31844.8350
1      1         1     10  2020  4722.9800   38964.5850
2      1         1     11  2020  2842.6800   23452.1100
3      1         1     12  2020  1348.5000   11125.1250
4      1         2      1  2021  2247.9900   18545.9175
..   ...       ...    ...   ...        ...          ...
280    5         2      1  2025   135.2225       0.0000
281    5         3      2  2025   114.8400       0.0000
282    5         3      3  2025   125.6800       0.0000
283    5         3      4  2025   122.9750       0.0000
284    5         3      5  2025    88.1975       0.0000

[285 rows x 6 columns]


In [261]:
missing_values = linear_data.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [262]:
df_2025 = linear_data[linear_data['Year'] == 2025]
df_2025


,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,896.5525,0.0
53,1,3,2,2025,797.3625,0.0
54,1,3,3,2025,839.6650,0.0
55,1,3,4,2025,756.6600,0.0
56,1,3,5,2025,665.7075,0.0
109,2,2,1,2025,701.2000,0.0
110,2,3,2,2025,747.4500,0.0
111,2,3,3,2025,613.3475,0.0
112,2,3,4,2025,536.5900,0.0
113,2,3,5,2025,451.1075,0.0


In [263]:
df_2024 = linear_data[linear_data['Year'] == 2024]
df_2024

,DEP,Semester,Month,Year,Hours,Money Spent
40,1,2,1,2024,948.630000,8774.8275
41,1,3,2,2024,1057.590000,9782.7075
42,1,3,3,2024,694.670000,6425.6975
43,1,3,4,2024,1065.520000,9856.0600
44,1,3,5,2024,418.670000,3872.6975
45,1,4,6,2024,0.000000,0.0000
46,1,4,7,2024,0.000000,0.0000
47,1,4,8,2024,0.000000,0.0000
48,1,1,9,2024,1461.137500,0.0000
49,1,1,10,2024,1461.137500,0.0000


In [264]:
df_2025 = linear_data[linear_data['Year'] == 2025]
df_2025

,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,896.5525,0.0
53,1,3,2,2025,797.3625,0.0
54,1,3,3,2025,839.6650,0.0
55,1,3,4,2025,756.6600,0.0
56,1,3,5,2025,665.7075,0.0
109,2,2,1,2025,701.2000,0.0
110,2,3,2,2025,747.4500,0.0
111,2,3,3,2025,613.3475,0.0
112,2,3,4,2025,536.5900,0.0
113,2,3,5,2025,451.1075,0.0


In [265]:
PredictedData = linear_data.copy()


In [266]:
PredictedData

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.9800,31844.8350
1,1,1,10,2020,4722.9800,38964.5850
2,1,1,11,2020,2842.6800,23452.1100
3,1,1,12,2020,1348.5000,11125.1250
4,1,2,1,2021,2247.9900,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,135.2225,0.0000
281,5,3,2,2025,114.8400,0.0000
282,5,3,3,2025,125.6800,0.0000
283,5,3,4,2025,122.9750,0.0000


In [267]:
PredictedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        285 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [268]:
missing_values = PredictedData.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [7]:
DinningPred2025 = pd.read_excel("DinningData.xlsx", sheet_name='Academic Year 24 25 ')
DinningPred2025

,DEP,Semester,Month,Academic Year,Hours,Money Spent
0,Res Dining,Fall,Sep,24/25,1461.137500,13515.521875
1,Res Dining,Fall,Oct,24/25,1461.137500,13515.521875
2,Res Dining,Fall,Nov,24/25,956.562500,8848.203125
3,Res Dining,Fall,Dec,24/25,993.367500,9188.649375
4,Res Dining,J-Term,Jan,24/25,896.552500,8293.110625
5,Res Dining,Spring,Feb,24/25,797.362500,7375.603125
6,Res Dining,Spring,Mar,24/25,839.665000,7766.901250
7,Res Dining,Spring,Apr,24/25,756.660000,6999.105000
8,Res Dining,Spring,May,24/25,665.707500,6157.794375
9,Martys,Fall,Sep,24/25,1170.652500,10828.535625


In [10]:
agg_data = DinningPred2025.groupby(['DEP', 'Month'], as_index=False)['Money Spent'].sum()
agg_data

,DEP,Month,Money Spent
0,C-Store,Apr,1998.346875
1,C-Store,Dec,2326.883750
2,C-Store,Feb,2677.805625
3,C-Store,Jan,2313.910625
4,C-Store,Mar,1746.053125
5,C-Store,May,1294.791875
6,C-Store,Nov,3140.791250
7,C-Store,Oct,3022.229375
8,C-Store,Sep,3995.375625
9,Catering,Apr,1137.518750


In [24]:
color_map = {
    'Res Dining': '#008080',   # Teal
    'Martys': '#FF7F50',       # Coral
    'Oneota': '#DAA520',       # Goldenrod
    'C-Store': '#6A5ACD',      # Slate Blue
    'Catering': '#2E8B57'      # Sea Green
}

month_order = ['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']

fig = px.bar(agg_data, 
             x='Month', 
             y='Money Spent', 
             color='DEP', 
             title='Total Money Spent by Department per Semester',
             labels={'Money Spent': 'Total Money Spent ($)', 'DEP': 'Department'},
             barmode='group',
             color_discrete_map=color_map,
             category_orders={'Month': month_order},
             facet_col='DEP',  # Create a separate subplot for each department
             facet_col_wrap=2)  # Number of columns in the facet grid

# Show the plot
fig.show()

In [12]:
fig = px.bar(agg_data, x='DEP', y='Money Spent', color='Month', 
             title='Total Money Spent by Department and Semester', 
             labels={'Money Spent': 'Money Spent', 'DEP': 'Department'},
             text_auto=True)

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [13]:
fig = px.treemap(agg_data, path=['DEP', 'Month'], values='Money Spent', 
                 title='Treemap of Money Spent by Department and Semester', 
                 labels={'Money Spent': 'Money Spent', 'DEP': 'Department'})

fig.show()